In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *
import datetime
import tqdm

In [2]:
train_data = read_data('train')
test_data = read_data('test')

train_y = read_target_train()

## Получение данных о продолжительности различных операций

In [2]:
def cat_preprocess(df, drop_cols, encode_col):
    df = df.drop(drop_cols, axis = 1)
    df = pd.get_dummies(df, columns=[encode_col])
    return df

In [5]:
def cat_feat_extr(df, cols, ves_col):
    for NPLV in tqdm(df.NPLV.unique()):
        try:
            for col in cols:
                if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                    for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                        df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
        except:
            try:
                for col in cols:
                    if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                        for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                            df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                    df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
            except:
                try:
                    for col in cols:
                        if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                            for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                                df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                        df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
                except:
                    for col in cols:
                        if not df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()].empty: 
                            for i in range(len(df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()])):
                                df.at[df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i], col] = \
                                        df[ves_col][df.index[(df['NPLV'] == NPLV) & (df[col] > 0)].to_list()[i]]
                    
    df = df.drop([ves_col], axis = 1).groupby('NPLV').agg('sum')
    
    return df

хронометраж - время начала и конца различных операций во время плавки

In [9]:
train_data['chronom'].head()

,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,510008,межпл.прост.,межпл. простой,2011-01-01 10:28:13,2011-01-01 10:32:15,NaN
1,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN


In [10]:
test_data['chronom'].head()

,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,512324,межпл.прост.,межпл. простой,2011-05-07 21:28:59,2011-05-07 21:33:38,NaN
1,512324,межпл.прост.,межпл. простой,2021-05-05 17:35:55,2021-05-05 17:41:21,NaN
2,512324,межпл.прост.,Осмотр конвертера,2021-05-05 17:36:07,2021-05-05 17:39:07,NaN
3,512324,межпл.прост.,Слив шлака,2021-05-05 17:39:07,2021-05-05 17:41:20,NaN
4,512324,опер,Завалка лома,2021-05-05 17:41:21,2021-05-05 17:43:16,NaN


In [11]:
train_data['chronom']['VR_NACH'] = pd.to_datetime(train_data['chronom']['VR_NACH'])
train_data['chronom']['VR_KON'] = pd.to_datetime(train_data['chronom']['VR_KON'])
train_data['chronom']['duration'] = (train_data['chronom']['VR_KON'] - train_data['chronom']['VR_NACH']).dt.total_seconds()

test_data['chronom']['VR_NACH'] = pd.to_datetime(test_data['chronom']['VR_NACH'])
test_data['chronom']['VR_KON'] = pd.to_datetime(test_data['chronom']['VR_KON'])
test_data['chronom']['duration'] = (test_data['chronom']['VR_KON'] - test_data['chronom']['VR_NACH']).dt.total_seconds()

train_data['chronom'].head()

,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2,duration
0,510008,межпл.прост.,межпл. простой,2011-01-01 10:28:13,2011-01-01 10:32:15,NaN,242.0
1,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN,424.0
2,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN,144.0
3,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN,100.0
4,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN,46.0


In [12]:
names_train_nop = train_data['chronom']['NOP'].value_counts()
names_train_nop_stay = set(names_train_nop[names_train_nop > 100].to_dict().keys())
names_train_nop = set(names_train_nop.to_dict().keys())

names_test_nop = set(test_data['chronom']['NOP'].value_counts().to_dict().keys())

name_to_stay = names_train_nop_stay.intersection(names_test_nop)

In [13]:
names_to_del_train = names_train_nop.difference(name_to_stay)
names_to_del_test = names_test_nop.difference(name_to_stay)

In [14]:
prep_chronom_train = cat_preprocess(train_data['chronom'], ['TYPE_OPER', 'VR_NACH', 'VR_KON', 'O2'], 'NOP')
prep_chronom_test = cat_preprocess(test_data['chronom'], ['TYPE_OPER', 'VR_NACH', 'VR_KON', 'O2'], 'NOP')

In [15]:
prep_chronom_train_cols = list(prep_chronom_train.columns.drop(['NPLV', 'duration']))
prep_chronom_test_cols = list(prep_chronom_test.columns.drop(['NPLV', 'duration']))

In [16]:
prep_chronom_train = prep_chronom_train.fillna(0)
prep_chronom_test = prep_chronom_test.fillna(0)

prep_chronom_train['duration'] = prep_chronom_train['duration'].astype('int64')
prep_chronom_test['duration'] = prep_chronom_test['duration'].astype('int64')

In [17]:
prep_chronom_train[prep_chronom_train_cols] = prep_chronom_train[prep_chronom_train_cols].astype('int64')
prep_chronom_test[prep_chronom_test_cols] = prep_chronom_test[prep_chronom_test_cols].astype('int64')

In [22]:
cat_chronom_train = cat_feat_extr(prep_chronom_train, prep_chronom_train_cols, 'duration')
cat_chronom_test = cat_feat_extr(prep_chronom_test, prep_chronom_test_cols, 'duration')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 780/780 [00:18<00:00, 41.55it/s]


In [23]:
for col in names_to_del_train:
    col_ = 'NOP_' + col
    del cat_chronom_train[col_]

for col in names_to_del_test:
    col_ = 'NOP_' + col
    del cat_chronom_test[col_]

In [26]:
cat_chronom_train.to_csv('data_chronom/chronom_duration_train.csv')
cat_chronom_test.to_csv('data_chronom/chronom_duration_test.csv')

## Конец получения фичей chronom_duration

### Фиксируем количество газов во время начала и конца продувки, можно запускать независимо от предыдущей части

In [3]:
cat_chronom_train = pd.read_csv('data_chronom/chronom_duration_train.csv')
cat_chronom_test = pd.read_csv('data_chronom/chronom_duration_test.csv')

cat_chronom_train = cat_chronom_train.set_index('NPLV')
cat_chronom_test = cat_chronom_test.set_index('NPLV')

In [4]:
train_gas = pd.read_csv('data/gas_train.csv')
test_gas = pd.read_csv('data/gas_test.csv')

In [5]:
train_gas['Time'] = pd.to_datetime(train_gas['Time'])
test_gas['Time'] = pd.to_datetime(test_gas['Time'])

train_data['chronom']['VR_NACH'] = pd.to_datetime(train_data['chronom']['VR_NACH'])
test_data['chronom']['VR_NACH'] = pd.to_datetime(test_data['chronom']['VR_NACH'])
train_data['chronom']['VR_KON'] = pd.to_datetime(train_data['chronom']['VR_KON'])
test_data['chronom']['VR_KON'] = pd.to_datetime(test_data['chronom']['VR_KON'])

In [6]:
row = train_data['chronom'][train_data['chronom']['NOP'] == 'Продувка'].set_index('NPLV')
start_train_nach = row['VR_NACH']
end_train_nach = row['VR_KON']

row = test_data['chronom'][test_data['chronom']['NOP'] == 'Продувка'].set_index('NPLV')
start_test_nach = row['VR_NACH']
end_test_nach = row['VR_KON']

In [18]:
col_gas_feat = ['V', 'T', 'O2', 'N2', 'H2', 'CO2', 'CO', 'AR', 'O2_pressure']

type_oper = '_start_Продувка'
for col in col_gas_feat:
    cat_chronom_train[col + type_oper] = 0.0
    cat_chronom_test[col + type_oper] = 0.0
    
type_oper = '_end_Продувка'
for col in col_gas_feat:
    cat_chronom_train[col + type_oper] = 0.0
    cat_chronom_test[col + type_oper] = 0.0

In [28]:
%%time

for nplv_obj in tqdm.tqdm(train_gas['NPLV'].unique()):
    try:
        df_nplv = train_gas[(train_gas['NPLV'] == nplv_obj) & \
                            ((train_gas['Time'] - start_train_nach[nplv_obj]).dt.total_seconds() <= 1) & \
                            ((train_gas['Time'] - start_train_nach[nplv_obj]).dt.total_seconds() >= 0)]
        type_oper = '_start_Продувка'
        for gas_feat in col_gas_feat:
            cat_chronom_train.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat]
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████| 2063/2063 [14:36<00:00,  2.35it/s]

Wall time: 14min 36s


In [45]:
for nplv_obj in tqdm.tqdm(test_gas['NPLV'].unique()):
    try:
        df_nplv = test_gas[(test_gas['NPLV'] == nplv_obj) & \
                            ((test_gas['Time'] - start_test_nach[nplv_obj]).dt.total_seconds() <= 1) & \
                            ((test_gas['Time'] - start_test_nach[nplv_obj]).dt.total_seconds() >= 0)]
        type_oper = '_start_Продувка'
        for gas_feat in col_gas_feat:
            cat_chronom_test.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat] 
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████| 780/780 [02:14<00:00,  5.79it/s]


In [34]:
for nplv_obj in tqdm.tqdm(train_gas['NPLV'].unique()):
    try:
        df_nplv = train_gas[(train_gas['NPLV'] == nplv_obj) & \
                            ((end_train_nach[nplv_obj] - train_gas['Time']).dt.total_seconds() <= 1) & \
                            ((end_train_nach[nplv_obj] - train_gas['Time']).dt.total_seconds() >= 0)]
        type_oper = '_end_Продувка'
        for gas_feat in col_gas_feat:
            cat_chronom_train.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat]
    except:
        pass 

100%|███████████████████████████████████████████████████████████████████████████████████████| 2063/2063 [16:28<00:00,  2.09it/s]


In [47]:
for nplv_obj in tqdm.tqdm(test_gas['NPLV'].unique()):
    try:
        df_nplv = test_gas[(test_gas['NPLV'] == nplv_obj) & \
                            ((end_test_nach[nplv_obj] - test_gas['Time']).dt.total_seconds() <= 1) & \
                            ((end_test_nach[nplv_obj] - test_gas['Time']).dt.total_seconds() >= 0)]
        type_oper = '_end_Продувка'
        for gas_feat in col_gas_feat:
            cat_chronom_test.at[nplv_obj, gas_feat + type_oper] = df_nplv[gas_feat] 
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████| 780/780 [02:30<00:00,  5.17it/s]


In [49]:
cat_chronom_train.to_csv('data_chronom/chronom_duration_train_new.csv')
cat_chronom_test.to_csv('data_chronom/chronom_duration_test_new.csv')

In [48]:
cat_chronom_test

,NOP_Вхождение в гр.МНЛЗ,NOP_Завалка лома,NOP_Заливка чугуна,NOP_Наведение гарнисажа,NOP_Нагрев лома,NOP_Наложение продувки,NOP_Неиспр. электр. обор,NOP_Обрыв горловины,NOP_Осмотр конвертера,NOP_Отсутствие O2,...,O2_pressure_start_Продувка,V_end_Продувка,T_end_Продувка,O2_end_Продувка,N2_end_Продувка,H2_end_Продувка,CO2_end_Продувка,CO_end_Продувка,AR_end_Продувка,O2_pressure_end_Продувка
NPLV,,,,,,,,,,,,,,,,,,,,,
512324,0,115,143,0,730,396,0,0,180,0,...,14.771412,208499.937500,723.958313,11.980000,81.160004,0.03,5.370000,0.380000,1.07,13.346354
512327,0,114,578,0,0,0,0,0,191,0,...,15.798611,232836.765625,821.527771,0.000000,68.419998,0.09,26.129999,4.350000,1.01,16.355614
512328,0,2096,150,0,0,0,0,0,39,0,...,15.849248,219953.703125,806.944458,0.000000,35.970001,0.32,19.180000,43.880001,0.63,16.355614
512331,344,3015,221,0,0,0,0,2586,189,0,...,15.834780,218263.343750,789.930603,0.000000,70.150002,0.00,25.180000,3.670000,1.00,16.326677
512333,0,1478,199,0,0,0,0,0,0,0,...,15.697338,224536.562500,810.763916,4.684565,57.770000,0.01,26.110001,15.280000,0.83,16.341145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513369,0,225,289,134,732,0,0,0,125,0,...,15.776910,226180.031250,795.833313,6.040000,79.570000,0.19,13.080000,0.000000,1.13,16.247107
513370,0,158,371,144,746,0,0,0,103,0,...,15.733507,230136.828125,855.902771,0.000000,76.510002,0.27,22.129999,0.000000,1.10,16.304977
513371,615,131,356,137,736,0,0,0,66,0,...,15.668403,222152.750000,784.722229,9.410000,80.919998,0.16,8.270000,0.000000,1.24,16.037327
